# Anomaly Visualization Notebook

This notebook visualizes abnormal responses in energy consumption data. It highlights anomalies over time and, if available, shows clustering results to further inspect abnormal household behavior.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the feature-engineered data
data_path = '../data/processed/smart_meter_data_features.csv'
data = pd.read_csv(data_path)
print('Data loaded successfully')
data.head()

In [3]:
# Visualize anomalies over time if 'anomaly' and 'timestamp' columns exist
if 'anomaly' in data.columns and 'timestamp' in data.columns:
    # Convert timestamp column to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data.sort_values('timestamp', inplace=True)
    
    plt.figure(figsize=(14,7))
    
    # Plot overall energy consumption
    plt.plot(data['timestamp'], data['energy_consumption'], color='lightgray', label='Energy Consumption')
    
    # Highlight anomalies
    anomalies = data[data['anomaly'] == 1]
    plt.scatter(anomalies['timestamp'], anomalies['energy_consumption'], color='red', label='Anomaly', marker='o')
    
    plt.xlabel('Timestamp')
    plt.ylabel('Energy Consumption')
    plt.title('Energy Consumption Over Time with Anomalies Highlighted')
    plt.legend()
    plt.show()
else:
    print("Columns 'anomaly' and/or 'timestamp' not found in the dataset.")

In [4]:
# If clustering has been applied, visualize clusters
if 'cluster' in data.columns and 'energy_consumption' in data.columns and 'hour' in data.columns:
    plt.figure(figsize=(10,6))
    plt.scatter(data['hour'], data['energy_consumption'], c=data['cluster'], cmap='viridis', alpha=0.7)
    plt.xlabel('Hour of Day')
    plt.ylabel('Energy Consumption')
    plt.title('Energy Consumption vs. Hour Colored by Cluster')
    plt.colorbar(label='Cluster')
    plt.show()
else:
    print("Columns 'cluster', 'hour', or 'energy_consumption' not found for cluster visualization.")

## Conclusion

This notebook provides visual insights into the energy consumption patterns, highlighting anomalies that may indicate abnormal responses to dynamic pricing. Adjust the visualizations as needed for your analysis.